In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [16]:
# Alterar o nome do papel conforme necessário
ticker_name = "VALE3.SA"

In [24]:
# Carregar os dados do CSV para o histórico de mercado
ticker_history = pd.read_csv(f"./data/{ticker_name}.csv", header=[0, 1], index_col=0)
ticker_history.head()

Price,Close,High,Low,Open,Volume
Ticker,VALE3.SA,VALE3.SA,VALE3.SA,VALE3.SA,VALE3.SA
Date,,,,,
2020-01-02,32.831028,32.897498,32.480539,32.619525,17509700
2020-01-03,32.589310,32.933754,32.377809,32.438237,17284800
2020-01-06,32.395931,32.510745,32.117959,32.510745,32787800
2020-01-07,32.631607,32.746421,32.269035,32.329463,16326400
2020-01-08,32.637653,32.855198,32.401981,32.752470,15298500


In [21]:
# Carregar os dados do CSV para histórico de dividendos. A data marcada é equivalente à data EX, ou seja,
# o dia em que a ação sofreu correção em seu valor. Carregar em uma série por ser apenas uma coluna
ticker_dividend = pd.read_csv(f"./data/{ticker_name}_dividend.csv", index_col=0).squeeze()
ticker_dividend.tail()

Date
2023-08-14 00:00:00-03:00    1.918472
2024-03-12 00:00:00-03:00    2.738548
2024-08-05 00:00:00-03:00    2.093798
2024-12-12 00:00:00-03:00    0.520531
2025-03-10 00:00:00-03:00    2.141847
Name: Dividends, dtype: float64

# Encontrar curva de recuperação normalizada
Vamos utilizar algumas data com passadas, normalizar os dados de fechamento com o valor pago do dividendo e descobrir um padrão. Esse padrão vai ser necessário para prever o gráfico futuro após a próxima data com.

In [ ]:
# Juntar dados das últimas data com
total_dates = 5
count_days = 30 
curvas_normalizadas = []

for i in range(1, total_dates + 1):
    ex_date = pd.to_datetime(ticker_dividend.index[-i]).replace(tzinfo=None)
    valor_provento = ticker_dividend.iloc[-i]
    
    # Obter histórico de 30 dias após a data-ex
    start_date = ex_date
    end_date = ex_date + pd.Timedelta(days=count_days)
    historico = ticker_history[(pd.to_datetime(ticker_history.index) > start_date) & (pd.to_datetime(ticker_history.index) < end_date)]
    

    if len(historico) < count_days:
        print("pulou")
        continue  # pula se não tiver dados suficientes
        

    preco_base = historico.iloc[0]["Close"]
    curva_normalizada = historico["Close"].iloc[:count_days] / preco_base
    curva_normalizada.index = range(count_days)  # reinicia índice como dias
    curvas_normalizadas.append(curva_normalizada)



Price   Ticker  
Close   VALE3.SA    5.444000e+01
High    VALE3.SA    5.472000e+01
Low     VALE3.SA    5.341000e+01
Open    VALE3.SA    5.408000e+01
Volume  VALE3.SA    2.306010e+07
Name: 2025-03-11, dtype: float64
Price   Ticker  
Close   VALE3.SA    5.376173e+01
High    VALE3.SA    5.457980e+01
Low     VALE3.SA    5.360774e+01
Open    VALE3.SA    5.435844e+01
Volume  VALE3.SA    1.853280e+07
Name: 2024-12-13, dtype: float64
Price   Ticker  
Close   VALE3.SA    5.466931e+01
High    VALE3.SA    5.486963e+01
Low     VALE3.SA    5.370585e+01
Open    VALE3.SA    5.412558e+01
Volume  VALE3.SA    2.393020e+07
Name: 2024-08-06, dtype: float64
Price   Ticker  
Close   VALE3.SA    5.635018e+01
High    VALE3.SA    5.649743e+01
Low     VALE3.SA    5.548509e+01
Open    VALE3.SA    5.577039e+01
Volume  VALE3.SA    3.276920e+07
Name: 2024-03-13, dtype: float64
Price   Ticker  
Close   VALE3.SA    5.237430e+01
High    VALE3.SA    5.294683e+01
Low     VALE3.SA    5.224612e+01
Open    VALE3.SA    5.29

In [ ]:
# Obter histórico de preços ao redor da data-ex
inicio = last_data_ex - pd.Timedelta(days=10)
fim = last_data_ex + pd.Timedelta(days=15)
ticker_history[(pd.to_datetime(ticker_history.index) > inicio) & (pd.to_datetime(ticker_history.index) < fim)]

Price,Close,High,Low,Open,Volume
Ticker,VALE3.SA,VALE3.SA,VALE3.SA,VALE3.SA,VALE3.SA
Date,,,,,
2025-03-05,53.501869,54.108205,53.309381,54.040835,13843400
2025-03-06,54.088955,54.396934,53.463369,53.549989,17678400
2025-03-07,54.878155,55.195757,53.463371,53.877219,25480600
2025-03-10,53.990002,54.549999,53.250000,54.230000,23626600
2025-03-11,54.439999,54.720001,53.410000,54.080002,23060100
2025-03-12,53.759998,54.500000,53.150002,54.500000,18825700
2025-03-13,54.500000,55.160000,53.700001,53.880001,20584600
2025-03-14,56.290001,56.410000,55.160000,55.200001,31789300


In [15]:
# Conferir matriz de correlação entre variáveis independentes
print(vale[['Open', 'High', 'Low', 'Volume']].corr())

Price                Open      High       Low    Volume
Ticker           VALE3.SA  VALE3.SA  VALE3.SA  VALE3.SA
Price  Ticker                                          
Open   VALE3.SA  1.000000  0.998869  0.998884 -0.130650
High   VALE3.SA  0.998869  1.000000  0.998657 -0.116871
Low    VALE3.SA  0.998884  0.998657  1.000000 -0.141556
Volume VALE3.SA -0.130650 -0.116871 -0.141556  1.000000


In [ ]:
vale['Label'] = vale['Close'].shift(-1)
vale.dropna(inplace=True)

X = vale[['Open', 'High', 'Low']]
y = vale['Label'].astype(float)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

k_values = range(1, 21)
rmse_scores = []

for k in k_values:
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred_knn = knn.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_knn))  # Calcular o RMSE
    rmse_scores.append(rmse)

best_k = k_values[np.argmin(rmse_scores)]
print(f"Melhor valor de k: {best_k}")
print(f"RMSE para os diferentes valores de k: {rmse_scores}")

knn_best = KNeighborsRegressor(n_neighbors=best_k)
knn_best.fit(X_train, y_train)
y_pred_knn_best = knn_best.predict(X_test)

def avaliar(y_test, y_pred, nome_modelo):
    print(f"\n=== {nome_modelo} ===")
    print("MAE:", mean_absolute_error(y_test, y_pred))
    print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
    print("R²:", r2_score(y_test, y_pred))

avaliar(y_test, y_pred_lr, "Regressão Linear")
avaliar(y_test, y_pred_knn_best, "KNN (Melhor k)")

num_amostras = len(y_test)

plt.figure(figsize=(10, 5))
plt.plot(range(num_amostras), y_test.values[:num_amostras], label="Real", marker='o')
plt.plot(range(num_amostras), y_pred_lr[:num_amostras], label="Reg. Linear", marker='x')
plt.plot(range(num_amostras), y_pred_knn_best[:num_amostras], label=f"KNN (k={best_k})", marker='s')
plt.title("Comparação dos modelos (Primeiras amostras)")
plt.xlabel("Amostras")
plt.ylabel("Preço de Fechamento")
plt.legend()
plt.grid(True)
plt.show()


TypeError: unsupported operand type(s) for /: 'str' and 'int'